# Covid-19 Издөө маалыматтарды талдоо [Data Analysis ]

## Covid-19 Маалыматтар базасы 

In [ ]:
#!pip install folium
#!pip install plotly

In [ ]:
# imports
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import folium


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import math
import random
from datetime import timedelta

import warnings
warnings.filterwarnings('ignore')

#color pallette
cnf = '#393e46'
dth = '#ff2e63'
rec = '#21bf73'
act = '#fe9801'

# Маалыматтар базасын даярдоо

In [ ]:
import plotly as py
py.offline.init_notebook_mode(connected = True)

In [ ]:
import os

In [ ]:
try:
    os.system("rm -rf Covid-19-Preprocessed-Dataset")
except:
    print('File does not exist')

In [ ]:
#!git clone https://github.com/laxmimerit/Covid-19-Preprocessed-Dataset.git

In [ ]:
df = pd.read_csv('Covid-19-Preprocessed-Dataset-main/preprocessed/covid_19_data_cleaned.csv', parse_dates=['Date'])

country_daywise = pd.read_csv('Covid-19-Preprocessed-Dataset-main/preprocessed/country_daywise.csv', parse_dates=['Date'])
countywise = pd.read_csv('Covid-19-Preprocessed-Dataset-main/preprocessed/countrywise.csv')
daywise = pd.read_csv('Covid-19-Preprocessed-Dataset-main/preprocessed/daywise.csv', parse_dates=['Date'])


In [ ]:
df['Province/State'] = df['Province/State'].fillna("")
df.head()

In [ ]:
confirmed = df.groupby('Date').sum()['Confirmed'].reset_index()
recovered = df.groupby('Date').sum()['Recovered'].reset_index()
deaths = df.groupby('Date').sum()['Deaths'].reset_index()
deaths.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.query('Country == "US"')

# Дүйнө жүзү боюнча *тастыкталган*, *айыгып кеткен* жана *каза болгондордун* жалпы саны 

In [ ]:
confirmed.tail()

In [ ]:
recovered.tail()

In [ ]:
deaths.tail()

In [ ]:
# Import the necessaries libraries
import plotly.offline as pyo
# Set notebook mode to work in offline
pyo.init_notebook_mode()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = confirmed['Date'], y = confirmed['Confirmed'], mode = 'lines+markers', name = 'Тастыкталган', line = dict(color = "Orange", width = 2)))
fig.add_trace(go.Scatter(x = recovered['Date'], y = recovered['Recovered'], mode = 'lines+markers', name = 'Айыгып кеткен', line = dict(color = "Green", width = 2)))
fig.add_trace(go.Scatter(x = deaths['Date'], y = deaths['Deaths'], mode = 'lines+markers', name = 'Каза болгон', line = dict(color = "Red", width = 2)))
fig.update_layout(title = 'Дүйнө жүзүндөгү COVID-19 учурлары', xaxis_tickfont_size = 14, yaxis = dict(title = 'Учурлардын саны'))

fig.show()

# Дүйнөлүк картадагы учурлардын жыштыгынын анимациясы

In [ ]:
df.info()

In [ ]:
df['Date'] = df['Date'].astype(str)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
fig = px.density_mapbox(df, lat = 'Lat', lon = 'Long', hover_name = 'Country', hover_data = ['Confirmed', 'Recovered', 'Deaths'], animation_frame='Date', color_continuous_scale='Portland', radius = 7, zoom = 0, height= 700)
fig.update_layout(title = 'Убакыттын өтүшү менен дүйнө жүзү боюнча COVID-19 учурлары')
fig.update_layout(mapbox_style = 'open-street-map', mapbox_center_lon = 0)


fig.show()

# Total Cases on Ships 

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.info()

In [ ]:
# Ships
# =====================

ship_rows = df['Province/State'].str.contains('Grand Princess') | df['Province/State'].str.contains('Diamond Princess') | df['Country'].str.contains('Grand Princess') | df['Country'].str.contains('Diamond Princess') | df['Country'].str.contains('MS Zaandam')
ship = df[ship_rows]

df = df[~ship_rows]

In [ ]:
ship_latest = ship[ship['Date'] == max(ship['Date'])]
ship_latest

In [ ]:
ship_latest.style.background_gradient(cmap = 'Pastel1_r')

# Убакыттын өтүшү менен аянты

In [ ]:
temp = df.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
temp = temp[temp['Date']==max(temp['Date'])].reset_index(drop = True)

tm = temp.melt(id_vars = 'Date', value_vars = ['Active', 'Deaths', 'Recovered'])
fig = px.treemap(tm, path = ['variable'], values = 'value', height = 250, width = 800, color_discrete_sequence=[act, rec, dth])

fig.data[0].textinfo = 'label+text+value'
fig.show()

In [ ]:
temp = df.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()
temp = temp.melt(id_vars = 'Date', value_vars = ['Recovered', 'Deaths', 'Active'], var_name = 'Case', value_name = 'Count')

fig = px.area(temp, x = 'Date', y = 'Count', color= 'Case', height = 400, title = 'Убакыттын өтүшү менен', color_discrete_sequence=[rec, dth,  act])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

# Folium Maps - картасы

In [ ]:
# Folium Maps'те Дүйнө жүзүндөгү COVID-19 учурлары

In [ ]:
temp = df[df['Date']==max(df['Date'])]

m = folium.Map(location=[0, 0], tiles='cartodbpositron', min_zoom = 1, max_zoom=4, zoom_start=1)

for i in range(0, len(temp)):
    folium.Circle(location=[temp.iloc[i]['Lat'], temp.iloc[i]['Long']], color = 'crimson', fill = 'crimson',
                 tooltip = '<li><bold> Country: ' + str(temp.iloc[i]['Country'])+
                            '<li><bold> Province: ' + str(temp.iloc[i]['Province/State'])+
                             '<li><bold> Confirmed: ' + str(temp.iloc[i]['Confirmed'])+
                             '<li><bold> Deaths: ' + str(temp.iloc[i]['Deaths']),
                 radius = int(temp.iloc[i]['Confirmed'])**0.5).add_to(m)
    

m

# Choropleth Map менен аныкталган учурлар

In [ ]:
country_daywise.head()

In [ ]:
fig = px.choropleth(country_daywise, locations= 'Country', locationmode='country names', color = np.log(country_daywise['Confirmed']),
                   hover_name = 'Country', animation_frame=country_daywise['Date'].dt.strftime('%Y-%m-%d'),
                   title='Убакыттын өтүшү менен учурлар көрсөтүлдү', color_continuous_scale=px.colors.sequential.Inferno)

fig.update(layout_coloraxis_showscale = True)
fig.show()

In [ ]:
fig = px.choropleth(country_daywise, locations= 'Country', locationmode='country names', color = country_daywise['Confirmed'],
                   hover_name = 'Country', animation_frame=country_daywise['Date'].dt.strftime('%Y-%m-%d'),
                   title='Убакыттын өтүшү менен учурлар', color_continuous_scale=px.colors.sequential.Inferno)

fig.update(layout_coloraxis_showscale = True)
fig.show()

# Аныкталган жана Каза болгон учурлар Static Colormap менен

In [ ]:
fig_c = px.choropleth(countywise, locations='Country', locationmode='country names',
                     color = np.log(countywise['Confirmed']), hover_name = 'Country',
                     hover_data = ['Confirmed'])

temp = countywise[countywise['Deaths']>0]
fig_d = px.choropleth(temp, locations='Country', locationmode='country names',
                     color = np.log(temp['Deaths']), hover_name = 'Country',
                     hover_data = ['Deaths'])

fig = make_subplots(rows = 1, cols = 2, subplot_titles= ['Аныкталган', 'Каза болгон'],
                   specs = [[{'type': 'choropleth'}, {'type': 'choropleth'}]])

fig.add_trace(fig_c['data'][0], row = 1, col = 1)
fig.add_trace(fig_d['data'][0], row = 1, col = 2)

fig.update(layout_coloraxis_showscale = False)

fig.show()

# Каза болгон жана Айыгып кеткен ар бир 100 учуру

In [ ]:
daywise.head()

In [ ]:
fig_c = px.bar(daywise, x = 'Date', y = 'Confirmed', color_discrete_sequence=[act])
fig_d = px.bar(daywise, x = 'Date', y = 'Deaths', color_discrete_sequence=[dth])

fig = make_subplots(rows = 1, cols = 2, shared_xaxes=False, horizontal_spacing=0.1,
                   subplot_titles=('Аныкталган учуру', 'Каза болгон учуру'))

fig.add_trace(fig_c['data'][0], row = 1, col = 1)
fig.add_trace(fig_d['data'][0], row = 1, col = 2)

fig.update_layout(height = 400)

fig.show()


In [ ]:
daywise.columns

In [ ]:
fig1 = px.line(daywise, x = 'Date', y = 'Deaths / 100 Cases', color_discrete_sequence=[dth])
fig2 = px.line(daywise, x = 'Date', y = 'Recovered / 100 Cases', color_discrete_sequence=[rec])
fig3 = px.line(daywise, x = 'Date', y = 'Deaths / 100 Recovered', color_discrete_sequence=['aqua'])

fig = make_subplots(rows = 1, cols = 3, shared_xaxes=False,
                   subplot_titles=('Каза болгон / 100 учур', 'Айыгып кеткен / 100 учур', 'Каза болгон / 100 Айыгып кеткен'))

fig.add_trace(fig1['data'][0], row = 1, col = 1)
fig.add_trace(fig2['data'][0], row = 1, col = 2)
fig.add_trace(fig3['data'][0], row = 1, col = 3)

fig.update_layout(height = 400)
fig.show()

# Жаңы учурлардын жана Өлкөлөрдүн саны

In [ ]:
fig_c = px.bar(daywise, x = 'Date', y = 'Confirmed', color_discrete_sequence=[act])
fig_d = px.bar(daywise, x = 'Date', y = 'No. of Countries', color_discrete_sequence=[dth])

fig = make_subplots(rows = 1, cols = 2, shared_xaxes=False, horizontal_spacing=0.1,
                   subplot_titles=('Жаңы учурлардын күндүк саны', 'Өлкөлөрдүн саны'))

fig.add_trace(fig_c['data'][0], row = 1, col = 1)
fig.add_trace(fig_d['data'][0], row = 1, col = 2)

fig.show()

# TOP 15 Өлкөлөрдүн анализдөөсү

In [ ]:
countywise.columns

In [ ]:
top  = 15

fig_c = px.bar(countywise.sort_values('Confirmed').tail(top), x = 'Confirmed', y = 'Country',
              text = 'Confirmed', orientation='h', color_discrete_sequence=[act])
fig_d = px.bar(countywise.sort_values('Deaths').tail(top), x = 'Deaths', y = 'Country',
              text = 'Deaths', orientation='h', color_discrete_sequence=[dth])


fig_a = px.bar(countywise.sort_values('Active').tail(top), x = 'Active', y = 'Country',
              text = 'Active', orientation='h', color_discrete_sequence=['#434343'])
fig_r = px.bar(countywise.sort_values('Recovered').tail(top), x = 'Recovered', y = 'Country',
              text = 'Recovered', orientation='h', color_discrete_sequence=[rec])


fig_dc = px.bar(countywise.sort_values('Deaths / 100 Cases').tail(top), x = 'Deaths / 100 Cases', y = 'Country',
              text = 'Deaths / 100 Cases', orientation='h', color_discrete_sequence=['#f84351'])
fig_rc = px.bar(countywise.sort_values('Recovered / 100 Cases').tail(top), x = 'Recovered / 100 Cases', y = 'Country',
              text = 'Recovered / 100 Cases', orientation='h', color_discrete_sequence=['#a45398'])


fig_nc = px.bar(countywise.sort_values('New Cases').tail(top), x = 'New Cases', y = 'Country',
              text = 'New Cases', orientation='h', color_discrete_sequence=['#f04341'])
temp = countywise[countywise['Population']>1000000]
fig_p = px.bar(temp.sort_values('Cases / Million People').tail(top), x = 'Cases / Million People', y = 'Country',
              text = 'Cases / Million People', orientation='h', color_discrete_sequence=['#b40398'])



fig_wc = px.bar(countywise.sort_values('1 week change').tail(top), x = '1 week change', y = 'Country',
              text = '1 week change', orientation='h', color_discrete_sequence=['#c04041'])
temp = countywise[countywise['Confirmed']>100]
fig_wi = px.bar(temp.sort_values('1 week % increase').tail(top), x = '1 week % increase', y = 'Country',
              text = '1 week % increase', orientation='h', color_discrete_sequence=['#b00398'])


fig = make_subplots(rows = 5, cols = 2, shared_xaxes=False, horizontal_spacing=0.2, 
                    vertical_spacing=.05,
                   subplot_titles=('Тастыкталган учурлар', 'Каза болгон учурлар', 'Айыгып кеткен учурлар', 'Активдүү учурлар',
                                  'Каза болгон / 100 учур', 'Айыгып кеткен / 100 учур',
                                  'Жаңы учурлар', 'Учурлар / Миллион киши',
                                  '1 жумадагы өзгөрүш', '1 жумадагы % көбөйүү'))

fig.add_trace(fig_c['data'][0], row = 1, col = 1)
fig.add_trace(fig_d['data'][0], row = 1, col = 2)

fig.add_trace(fig_r['data'][0], row = 2, col = 1)
fig.add_trace(fig_a['data'][0], row = 2, col = 2)

fig.add_trace(fig_dc['data'][0], row = 3, col = 1)
fig.add_trace(fig_rc['data'][0], row = 3, col = 2)

fig.add_trace(fig_nc['data'][0], row = 4, col = 1)
fig.add_trace(fig_p['data'][0], row = 4, col = 2)

fig.add_trace(fig_wc['data'][0], row = 5, col = 1)
fig.add_trace(fig_wi['data'][0], row = 5, col = 2)

fig.update_layout(height = 3000)
fig.show()


# Static Plots Сактоо

In [ ]:
# install
# conda install -c plotly plotly-orca==1.2.1 psutil requests


In [ ]:
if not os.path.exists('images'):
    os.mkdir('images')

In [ ]:
fig.write_image('fig.png')

In [ ]:
fig.write_image('images/fig.jpeg')

In [ ]:
fig.write_image('images/fig.pdf')

# Scatter Plot Каза болгон жана Тактыкталган учурлар үчүн 

In [ ]:
top = 15
fig = px.scatter(countywise.sort_values('Deaths', ascending = False).head(top), 
                x = 'Confirmed', y = 'Deaths', color = 'Country', size = 'Confirmed', height = 600,
                text = 'Country', log_x = True, log_y = True, title='Каза болгон vs Такталган учурлар (учурлар log10 масштабында)')

fig.update_traces(textposition = 'top center')
fig.update_layout(showlegend = False)
fig.update_layout(xaxis_rangeslider_visible = True)
fig.show()

In [ ]:
countywise.sort_values('Deaths', ascending = False).head(15)

# Тастыкталган, Каза болгон, Жаңы учурлар vs Өлкө жана Убакыт

## Bar Plot

In [ ]:
fig = px.bar(country_daywise, x = 'Date', y = 'Confirmed', color = 'Country', height = 600,
            title='Тастыкталган', color_discrete_sequence=px.colors.cyclical.mygbm)
fig.show()

In [ ]:
fig = px.bar(country_daywise, x = 'Date', y = 'Deaths', color = 'Country', height = 600,
            title='Каза болгон', color_discrete_sequence=px.colors.cyclical.mygbm)
fig.show()

In [ ]:
fig = px.bar(country_daywise, x = 'Date', y = 'Recovered', color = 'Country', height = 600,
            title='Айыгып кеткен', color_discrete_sequence=px.colors.cyclical.mygbm)
fig.show()

In [ ]:
fig = px.bar(country_daywise, x = 'Date', y = 'New Cases', color = 'Country', height = 600,
            title='Жаңы учурлар', color_discrete_sequence=px.colors.cyclical.mygbm)
fig.show()

## Line Plot 

In [ ]:
fig = px.line(country_daywise, x = 'Date', y = 'Confirmed', color = 'Country', height = 600,
             title='Тастыкталган', color_discrete_sequence = px.colors.cyclical.mygbm)
fig.show()

fig = px.line(country_daywise, x = 'Date', y = 'Deaths', color = 'Country', height = 600,
             title='Каза болгон', color_discrete_sequence = px.colors.cyclical.mygbm)
fig.show()

fig = px.line(country_daywise, x = 'Date', y = 'Recovered', color = 'Country', height = 600,
             title='Айыккан', color_discrete_sequence = px.colors.cyclical.mygbm)
fig.show()

# 100 учурдан кийин өсүү темпи

In [ ]:
df.head()

In [ ]:
gt_100 = country_daywise[country_daywise['Confirmed']>100]['Country'].unique()
temp = df[df['Country'].isin(gt_100)]

temp = temp.groupby(['Country', 'Date'])['Confirmed'].sum().reset_index()
temp = temp[temp['Confirmed']>100]


min_date = temp.groupby('Country')['Date'].min().reset_index()
min_date.columns = ['Country', 'Min Date']


from_100th_case = pd.merge(temp, min_date, on = 'Country')
from_100th_case['N days'] = (from_100th_case['Date'] - from_100th_case['Min Date']).dt.days

fig = px.line(from_100th_case, x = 'N days', y = 'Confirmed', color = 'Country', title = 'N days from 100 case', height = 600)
fig.show()

# 1000 учурдан кийин өсүү темпи

In [ ]:
gt_1000 = country_daywise[country_daywise['Confirmed']>1000]['Country'].unique()
temp = df[df['Country'].isin(gt_1000)]

temp = temp.groupby(['Country', 'Date'])['Confirmed'].sum().reset_index()
temp = temp[temp['Confirmed']>1000]


min_date = temp.groupby('Country')['Date'].min().reset_index()
min_date.columns = ['Country', 'Min Date']


from_1000th_case = pd.merge(temp, min_date, on = 'Country')
from_1000th_case['N days'] = (from_1000th_case['Date'] - from_1000th_case['Min Date']).dt.days

fig = px.line(from_1000th_case, x = 'N days', y = 'Confirmed', color = 'Country', title = 'N days from 1000 case', height = 600)
fig.show()

# 10,000 учурдан кийин өсүү темпи 

In [ ]:
gt_10000 = country_daywise[country_daywise['Confirmed']>10000]['Country'].unique()
temp = df[df['Country'].isin(gt_10000)]

temp = temp.groupby(['Country', 'Date'])['Confirmed'].sum().reset_index()
temp = temp[temp['Confirmed']>10000]


min_date = temp.groupby('Country')['Date'].min().reset_index()
min_date.columns = ['Country', 'Min Date']


from_10000th_case = pd.merge(temp, min_date, on = 'Country')
from_10000th_case['N days'] = (from_10000th_case['Date'] - from_10000th_case['Min Date']).dt.days

fig = px.line(from_10000th_case, x = 'N days', y = 'Confirmed', color = 'Country', title = 'N days from 10000 case', height = 600)
fig.show()

# 100,000 учурдан кийин өсүү темпи

In [ ]:
gt_100000 = country_daywise[country_daywise['Confirmed']>100000]['Country'].unique()
temp = df[df['Country'].isin(gt_100000)]

temp = temp.groupby(['Country', 'Date'])['Confirmed'].sum().reset_index()
temp = temp[temp['Confirmed']>100000]


min_date = temp.groupby('Country')['Date'].min().reset_index()
min_date.columns = ['Country', 'Min Date']


from_100000th_case = pd.merge(temp, min_date, on = 'Country')
from_100000th_case['N days'] = (from_100000th_case['Date'] - from_100000th_case['Min Date']).dt.days

fig = px.line(from_100000th_case, x = 'N days', y = 'Confirmed', color = 'Country', title = 'N days from 100000 case', height = 600)
fig.show()

# Tree Map Analysis

## Тастыкталган учур

In [ ]:
full_latest = df[df['Date'] == max(df['Date'])]

fig = px.treemap(full_latest.sort_values(by = 'Confirmed', ascending = False).reset_index(drop = True),
                path = ['Country', 'Province/State'], values = 'Confirmed', height = 700,
                title = 'Тастыкталган учурлардын саны',
                color_discrete_sequence = px.colors.qualitative.Dark2)

fig.data[0].textinfo = 'label+text+value'
fig.show()

## Каза болгон учур

In [ ]:
full_latest = df[df['Date'] == max(df['Date'])]

fig = px.treemap(full_latest.sort_values(by = 'Deaths', ascending = False).reset_index(drop = True),
                path = ['Country', 'Province/State'], values = 'Deaths', height = 700,
                title = 'Каза болгон учурлардын саны',
                color_discrete_sequence = px.colors.qualitative.Dark2)

fig.data[0].textinfo = 'label+text+value'
fig.show()

# Баштапкы жана Акыркы учурлар Report Time

In [ ]:
first_date = df[df['Confirmed']>0]
first_date = first_date.groupby('Country')['Date'].agg(['min']).reset_index()


last_date = df.groupby(['Country', 'Date'])['Confirmed', 'Deaths', 'Recovered']
last_date = last_date.sum().diff().reset_index()


mask = (last_date['Country'] != last_date['Country'].shift(1))

last_date.loc[mask, 'Confirmed'] = np.nan
last_date.loc[mask, 'Deaths'] = np.nan
last_date.loc[mask, 'Recovered'] = np.nan

last_date = last_date[last_date['Confirmed']>0]
last_date = last_date.groupby('Country')['Date'].agg(['max']).reset_index()


first_last = pd.concat([first_date, last_date['max']], axis = 1)
first_last['max'] = first_last['max'] + timedelta(days = 1)

first_last['Days'] = first_last['max'] - first_last['min']
first_last['Task'] = first_last['Country']

first_last.columns = ['Country', 'Start', 'Finish', 'Days', 'Task']

first_last = first_last.sort_values('Days')

colors = ['#' + ''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(first_last))]

fig = ff.create_gantt(first_last, index_col = 'Country', colors = colors, show_colorbar = False,
                     bar_width=0.2, showgrid_x = True, showgrid_y=True, height = 2500)

fig.show()

# Тастыкталган учурлар өлкө жана күн боюнча

In [ ]:
country_daywise.head()

In [ ]:
temp = country_daywise.groupby(['Country', 'Date'])['Confirmed'].sum().reset_index()
temp = temp[temp['Country'].isin(gt_10000)]

countries = temp['Country'].unique()

ncols = 3
nrows = math.ceil(len(countries)/ncols)

fig = make_subplots(rows=nrows, cols = ncols, shared_xaxes= False, subplot_titles=countries)

for ind, country in enumerate(countries):
    row = int((ind/ncols)+1)
    col = int((ind%ncols)+1)
    fig.add_trace(go.Bar(x = temp['Date'], y = temp.loc[temp['Country']==country, 'Confirmed'], name = country), row = row, col = col)
    
fig.update_layout(height=4000, title_text = 'Ар бир өлкөдө тастыкталган учурлар')
fig.update_layout(showlegend = False)
fig.show()

# COVID-19 жана башка ушул сыяктуу эпидемиялар

In [ ]:
full_latest_latest

In [ ]:
# Wikipedia Source

epidemics = pd.DataFrame({
    'epidemic' : ['COVID-19', 'ТОРС', 'EBOLA', 'MERS', 'H1N1'],
    'start_year' : [2019, 2002, 2013, 2012, 2009],
    'end_year' : [2022, 2004, 2016, 2020, 2010],
    'confirmed' : [full_latest['Confirmed'].sum(), 8422, 28646, 2519, 6724149],
    'deaths' : [full_latest['Deaths'].sum(), 813, 11323, 866, 19654]
})

epidemics['mortality'] = round((epidemics['deaths']/epidemics['confirmed'])*100, 2)

epidemics.head()

In [ ]:
temp = epidemics.melt(id_vars='epidemic', value_vars=['confirmed', 'deaths', 'mortality'],
                     var_name='Case', value_name='Value')

fig = px.bar(temp, x = 'epidemic', y = 'Value', color = 'epidemic', text = 'Value', facet_col = 'Case',
            color_discrete_sequence= px.colors.qualitative.Bold)

fig.update_traces(textposition='outside')
fig.update_layout(uniformtext_minsize = 8, uniformtext_mode = 'hide')
fig.update_yaxes(showticklabels = False)
fig.layout.yaxis2.update(matches = None)
fig.layout.yaxis3.update(matches = None)
fig.show()